# FlowerBot - Custom Vision Model Training and Deployment

#### Ensure Azure Custom Vision python SDK is installed

In [1]:
!pip install azure-cognitiveservices-vision-customvision

In [2]:
import os
import time
import uuid

from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials

#### Define identifiers used to access Azure Custom Vision services

In [3]:
TRAIN_ENDPOINT = "https://flowerbot.cognitiveservices.azure.com/"
training_key = "d6635d824b174b9a9f1f3d20609ce44f"
PREDICT_ENDPOINT = "https://flowerbot-prediction.cognitiveservices.azure.com/"
prediction_key = "76a6fbacaeb242818c613d1ee3bcb162"
prediction_resource_id = "/subscriptions/55f090ce-963c-4515-a48f-58da5dc82243/resourceGroups/flowerbot/providers/Microsoft.CognitiveServices/accounts/flowerbot-Prediction"

#### Define variables used to access Azure Custom Vision services

In [4]:
credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(TRAIN_ENDPOINT, credentials)
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(PREDICT_ENDPOINT, prediction_credentials)

#### Create a new Custom Vision project

In [5]:
# Find the object detection domain
object_detection_domain = next(domain for domain in trainer.get_domains() if domain.type == "ObjectDetection" and domain.name == "General")

# Append a UUID to avoid project name collisions.
publish_iteration_name = "flowerbot-" + str(uuid.uuid4())

print("Creating project", publish_iteration_name)
project = trainer.create_project(publish_iteration_name, domain_id=object_detection_domain.id)

Creating project flowerbot-c78977fc-25b7-4eb0-9678-dbe5d7b65a7a


In [6]:
print(project.id)

82d55c80-b09c-4fb6-afe7-d4a0ed2686a3


#### Register tags for the Custom Vision project

In [7]:
tag_names = [
#    'astilbe',
#    'bellflower',
#    'black_eyed_susan',
#    'calendula',
#    'california_poppy',
#    'carnation',
#    'common_daisy',
#    'coreopsis',
#    'daffodil',
#    'dandelion',
#    'iris',
#    'magnolia',
    'rose',
    'sunflower',
    'tulip',
#    'water_lily'
]

# reduced training set for viable training runtime

In [8]:
tag_id = {}
for tag in tag_names:
    tag_id[tag] = trainer.create_tag(project.id, tag)

#### Upload tagged JPEG images

The dataset being used is: https://www.kaggle.com/datasets/l3llff/flowers

In [9]:
IMAGE_PATH = '../flowers/'
IMAGE_MAX = 30 # images per tag

In [10]:
from os import walk

images = {}
for flower in tag_names:
    path = IMAGE_PATH + flower
    for (_, dirnames, filenames) in walk(path):
        images[flower] = filenames[0:IMAGE_MAX]

#### Upload the files to the project, associating each with a tag

These form the basis of our training data and associated the files with tags.

In [11]:
def upload_batch(images, count, total):
    print('Uploading %d images (%d/%d)' % (len(images), count, total))
    batch = ImageFileCreateBatch(images=images)
    result = trainer.create_images_from_files(project.id, batch)
    if result.is_batch_successful:
        print('Upload completed.')
    else:
        print("Image batch upload failed.")
        for image in result.images:
            print("Image status: ", image.status)
        exit(-1)

In [12]:
batch_size = 64
complete = False
count = 0
total = len(tag_names) * IMAGE_MAX

print("Adding", total, "images in batches of", batch_size)

tagged_images = []
for flower in images.keys():
    for imagefile in images[flower]:
        filename = IMAGE_PATH + flower + '/' + imagefile
        with open(filename, mode="rb") as image:
            tagid = [tag_id[flower].id]
            entry = ImageFileCreateEntry(name=imagefile, contents=image.read(), tag_ids=tagid)
            tagged_images.append(entry)
        count += 1
        if count % batch_size == 0:
            upload_batch(tagged_images, count, total)
            tagged_images.clear()

if len(tagged_images) > 0:
    upload_batch(tagged_images, count, total)

Adding 90 images in batches of 64
Uploading 64 images (64/90)
Upload completed.
Uploading 26 images (90/90)
Upload completed.


#### Train an iteration of the project, creating a new prediction model

In [16]:
print ("Training...")
iteration = trainer.train_project(project.id)
while (iteration.status != "Completed"):
    iteration = trainer.get_iteration(project.id, iteration.id)
    print ("Training status: " + iteration.status)
    time.sleep(60)

Training...
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Completed


#### The iteration is now trained, publish it to the project endpoint

In [17]:
trainer.publish_iteration(project.id, iteration.id, publish_iteration_name, prediction_resource_id)
print ("Done!")

Done!


## Test the Image Classification Model

In [18]:
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials

In [19]:
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(PREDICT_ENDPOINT, prediction_credentials)

#### Test the prediction service using an image file

In [27]:
import os
import time

print('Project', project.id, 'iteration', publish_iteration_name)

# Open the sample image and get back the prediction results.
with open(os.path.join("images", "sunflower.jpg"), mode="rb") as test_data:
    t0 = time.time()
    results = predictor.detect_image(project.id, publish_iteration_name, test_data)
    t1 = time.time()
    print('Prediction elapsed time:', t1 - t0)

# Display the results.    
for prediction in results.predictions:
    print("\t  ", prediction.tag_name, '-', prediction.probability)
    break  # just the best guess

Project 82d55c80-b09c-4fb6-afe7-d4a0ed2686a3 iteration flowerbot-c78977fc-25b7-4eb0-9678-dbe5d7b65a7a
Prediction elapsed time: 1.635308027267456
	   sunflower - 0.84498626
